In [1]:
import unittest
from abc import ABC
from enum import Enum

In [2]:
class BankAccount:
    OVERDRAFT_LIMIT = -500

    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        print(f'Deposited {amount}, balance = {self.balance}')

    def withdraw(self, amount):
        if self.balance - amount >= BankAccount.OVERDRAFT_LIMIT:
            self.balance -= amount
            print(f'Withdrew {amount}, balance = {self.balance}')
            return True
        return False

    def __str__(self):
        return f'Balance = {self.balance}'

In [3]:
# optional
class Command(ABC):
    def __init__(self):
        self.success = False
        
    def invoke(self):
        pass

    def undo(self):
        pass

In [4]:
class BankAccountCommand(Command):

    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1

    def __init__(self, account, action, amount):
        super().__init__()
        self.amount = amount
        self.action = action
        self.account = account
        # self.success = None

    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            self.success = True
        elif self.action == self.Action.WITHDRAW:
            self.success = self.account.withdraw(self.amount)

    def undo(self):
        if not self.success:
            return
        # strictly speaking this is not correct
        # (you don't undo a deposit by withdrawing)
        # but it works for this demo, so...
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)

In [5]:
# ba = BankAccount()
# cmd = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 100)
# cmd.invoke()
# print('After $100 deposit:', ba)

# cmd.undo()
# print('$100 deposit undone:', ba)

In [6]:
# illegal_cmd = BankAccountCommand(ba, BankAccountCommand.Action.WITHDRAW, 1000)
# illegal_cmd.invoke()
# print('After impossible withdrawal:', ba)
# illegal_cmd.undo()
# print('After undo:', ba)

In [7]:
# try using this before using MoneyTransferCommand!
class CompositeBankAccountCommand(Command, list):
    def __init__(self, items=[]):
        super().__init__()
        for i in items:
            self.append(i)

    def invoke(self):
        for x in self:
            x.invoke()

    def undo(self):
        for x in reversed(self):
            x.undo()

In [8]:
class MoneyTransferCommand(CompositeBankAccountCommand):
    def __init__(self, from_acct, to_acct, amount):
        super().__init__([
            BankAccountCommand(from_acct,
                               BankAccountCommand.Action.WITHDRAW,
                               amount),
            BankAccountCommand(to_acct,
                               BankAccountCommand.Action.DEPOSIT,
                               amount)])

    def invoke(self):
        ok = True
        for cmd in self:
            if ok:
                cmd.invoke()
                ok = cmd.success
            else:
                cmd.success = False
        self.success = ok


In [10]:
class TestSuite(unittest.TestCase):
    def test_composite_deposit(self):
        print('\n===test_composite_deposit===')
        ba = BankAccount()
        deposit1 = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 1000)
        deposit2 = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 500)
        composite = CompositeBankAccountCommand([deposit1, deposit2])
        composite.invoke()
        print(ba)
        composite.undo()
        print(ba)

    def test_transfer_fail(self):
        print('\n===test_transfer_fail===')
        ba1 = BankAccount(100)
        ba2 = BankAccount()

        # composite isn't so good because of failure
        amount = 1000  # try 1000: no transactions should happen
        wc = BankAccountCommand(ba1, BankAccountCommand.Action.WITHDRAW, amount)
        dc = BankAccountCommand(ba2, BankAccountCommand.Action.DEPOSIT, amount)

        transfer = CompositeBankAccountCommand([wc, dc])

        transfer.invoke()
        print('ba1:', ba1, 'ba2:', ba2)  # end up in incorrect state
        transfer.undo()
        print('ba1:', ba1, 'ba2:', ba2)

    def test_better_transfer(self):
        print('\n===test_better_transfer===')
        ba1 = BankAccount(100)
        ba2 = BankAccount()

        amount = 50

        transfer = MoneyTransferCommand(ba1, ba2, amount)
        transfer.invoke()
        print('ba1:', ba1, 'ba2:', ba2)
        transfer.undo()
        print('ba1:', ba1, 'ba2:', ba2)
        print(transfer.success)

unittest.main(argv=['ignored', '-v'], exit=False)

test_better_transfer (__main__.TestSuite) ... ok
test_composite_deposit (__main__.TestSuite) ... ok
test_transfer_fail (__main__.TestSuite) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK



===test_better_transfer===
Withdrew 50, balance = 50
Deposited 50, balance = 50
ba1: Balance = 50 ba2: Balance = 50
Withdrew 50, balance = 0
Deposited 50, balance = 100
ba1: Balance = 100 ba2: Balance = 0
True

===test_composite_deposit===
Deposited 1000, balance = 1000
Deposited 500, balance = 1500
Balance = 1500
Withdrew 500, balance = 1000
Withdrew 1000, balance = 0
Balance = 0

===test_transfer_fail===
Deposited 1000, balance = 1000
ba1: Balance = 100 ba2: Balance = 1000
Withdrew 1000, balance = 0
ba1: Balance = 100 ba2: Balance = 0
